<a href="https://colab.research.google.com/github/vinnik-dmitry07/Chatbot/blob/main/train_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
!pip install --quiet parlai

Sat Feb 27 20:15:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from pathlib import Path

GDRIVE_ROOT = Path('/content/drive/MyDrive/')
SAVE_DIR = GDRIVE_ROOT / 'chatbot_model'
DATA_DIR = GDRIVE_ROOT / 'chatbot_data'

In [3]:
from datetime import timedelta

EPISODE_DT = timedelta(minutes=3)  # change to split messages in separate dialogues if time delta is greater than EPISODE_DT
TRAIN_PART, TEST_PART, VALID_PART = 0.996, 0.002, 0.002

assert TRAIN_PART + TEST_PART + VALID_PART == 1

In [4]:
from google.colab import drive

drive.mount(str(GDRIVE_ROOT.parent))

Mounted at /content/drive


In [5]:
import json

with open(DATA_DIR / 'result.json', 'r', encoding='utf8') as f:
    raw_messages = json.load(f)['messages']

In [6]:
from datetime import datetime

filtered_messages = []
for msg in raw_messages:
    if (
            'from' in msg and
            'from_id' in msg and
            'mime_type' not in msg and
            msg['text'] and
            isinstance(msg['text'], str) and
            len(msg['text']) < 50
    ):
        msg1 = msg.copy()
        msg1['date'] = datetime.strptime(msg1['date'], '%Y-%m-%dT%H:%M:%S')
        filtered_messages.append(msg1)

In [7]:
import re

joined_messages = []
for i in range(len(filtered_messages)):
    alphanum_text = re.sub(r'[^A-Za-z0-9 ]+', '', filtered_messages[i]['text']).strip()
    if alphanum_text:
        if (    
                joined_messages and    
                filtered_messages[i - 1]['from_id'] == filtered_messages[i]['from_id'] and
                filtered_messages[i - 1]['date'] - filtered_messages[i]['date'] <= EPISODE_DT
        ):
            joined_messages[-1]['text'] += ' ' + alphanum_text
        else:
            new_message = filtered_messages[i].copy()
            new_message['text'] = alphanum_text
            joined_messages.append(new_message)

In [8]:
def partition(alist, indices):
    return [alist[a:b] for a, b in zip([0] + indices, indices + [None])]

In [9]:
def save_jsonl(messages, suffix, human_readable=False):
    time_diffs = [messages[i + 1]['date'] - messages[i]['date'] for i in range(len(messages) - 1)]
    split_positions = [i + 1 for i in range(len(time_diffs)) if time_diffs[i] > EPISODE_DT]
    episodes = partition(messages, split_positions)
    print(f'{suffix} episodes: {len(episodes)}, messages: {len(messages)}')

    with open(DATA_DIR / f'data_{suffix}.jsonl', 'w', **({'encoding': 'utf8'} if human_readable else {})) as outfile:
        for episode in episodes:
            dialog = [{'id': i % 2, 'text': msg['text']} for i, msg in enumerate(episode)]
            episode = {'dialog': [dialog]}
            json.dump(episode, outfile, **({'ensure_ascii': False} if human_readable else {}))
            outfile.write('\n')

In [10]:
import numpy as np

train, test, valid = np.split(joined_messages, [
    int(TRAIN_PART * len(joined_messages)),
    int((TRAIN_PART + TEST_PART) * len(joined_messages)),
])

save_jsonl(train, suffix='train')
save_jsonl(test, suffix='test')
save_jsonl(valid, suffix='valid')

train episodes: 424, messages: 754917
test episodes: 1, messages: 1516
valid episodes: 1, messages: 1516


In [11]:
import shutil
import subprocess
import time
import threading
from pathlib import Path


def dir_size_bytes(path):
    return int(subprocess.check_output(['du','--bytes', '--summarize', path]).split()[0].decode())

def check_chache(max_cache_size_gb=18, check_period_minutes=5):
    this_id = str(threading.get_ident())
    thread_path = Path('/threads')
    thread_path.mkdir(exist_ok=True)

    def threads_ids():
        return [str(p.name) for p in thread_path.iterdir() if p.is_file()]
    
    if not(threads_ids()):
        (thread_path / this_id).open(mode='w').close()
        while True:
            ids = threads_ids()
            if not (len(ids) == 1 and ids[0] == this_id):
                break

            print(f'Thread {this_id} is checking chache.')

            for cache_path in Path('/root/.config/Google/DriveFS').glob('**/content_cache'):
                chache_path_str = str(cache_path)
                chache_size_gb = dir_size_bytes(chache_path_str) / 10 ** 9
                if chache_size_gb > max_cache_size_gb:
                    print(f'Deleting {chache_path_str} with size {chache_size_gb} GB.')
                    shutil.rmtree(chache_path_str)
            time.sleep(check_period_minutes * 60)

In [12]:
# threading.Thread(target=check_chache).start()

import os

os.environ['SAVE_DIR'] = str(SAVE_DIR)
!rm --recursive --force $SAVE_DIR
!mkdir --parents $SAVE_DIR


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    task='jsonfile',
    jsonfile_datapath=str(DATA_DIR / 'data'),
    jsonfile_datatype_extension=True,

    model='transformer/generator',
    model_file=str(SAVE_DIR / 'model'),
    
    init_model='zoo:tutorial_transformer_generator/model',

    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    dict_file='zoo:tutorial_transformer_generator/model.dict',
    learn_positional_embeddings=True,
    
    lr=1e-5, optimizer='adam',
    warmup_updates=5000,
    validation_metric='ppl',
    validation_every_n_secs=60 * 60,  # running eval: valid
    # save_every_n_secs=60,  # saving model checkpoint

    batchsize=12, fp16=True, fp16_impl='mem_efficient',
    
    skip_generation=True,
    
    dynamic_batching='full',

    label_turns='both',  # https://parl.ai/docs/core/teachers.html#parlai.core.teachers.ConversationTeacher
)

20:19:02 | building data: /usr/local/lib/python3.7/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz
20:19:02 | Downloading http://parl.ai/downloads/_models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz to /usr/local/lib/python3.7/dist-packages/data/models/tutorial_transformer_generator/tutorial_transformer_generator_v1.tar.gz


20:19:37 | building dictionary first...
20:19:37 | No model with opt yet at: /content/drive/MyDrive/chatbot_model/model(.opt)
20:19:37 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,download_path: None,loglevel: info,dynamic_batching: full,verbose: False,datapath: /usr/local/lib/python3.7/dist-packages/data,eval_dynamic_batching: None,load_from_checkpoint: True,tensorboard_logdir: None,jsonfile_datapath: /content/drive/MyDrive/chatbot_data/data,jsonfile_datatype_extension: True,label_turns: both,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,beam_block_full_context: True,beam_length_penalty: 0.65,topk: 10,topp: 0.9,beam_delay: 30,beam_block_list_filename: None,temperature: 1.0,compute_tokenized_bleu: False,interactive_mode: False,fp16_impl: mem_efficient,force_fp16_tokens: False,adafactor_eps: (1e-30, 0.001),history_reversed: False,history_add_global_end_token: None,special_tok

/usr/local/lib/python3.7/dist-packages/parlai/utils/fp16.py:487: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


20:19:59 | Overflow: setting loss scale to 32768.0
20:20:00 | Overflow: setting loss scale to 16384.0
20:20:08 | time:10s total_exs:6492 epochs:0.01
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss        lr  ltpb  ltps   ppl  token_acc  \
   .9062  4241 13491 644.1 6492             19968  11.84    .4624 4.599 6.499e-08 882.7  2808 99.34      .2894   
    total_train_updates  tpb   tps   ups  
                     32 5123 16300 3.182

20:20:18 | time:20s total_exs:10012 epochs:0.01
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss       lr  ltpb  ltps   ppl  token_acc  \
       1  5769 20180 351.8 3520             16384  13.29    .3837 4.513 1.35e-07 456.3  1596 91.22      .3039   
    total_train_updates  tpb   tps   ups  
                     67 6226 21776 3.498

20:20:28 | time:31s total_exs:12752 epochs:0.02
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss       lr  ltpb  ltps   ppl  token_acc  \
       1  6114 214

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


22:11:03 | time:6666s total_exs:228844 epochs:0.30
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss    lr  ltpb  ltps   ppl  token_acc  \
       1  5916  8787 19.17  284             16384  20.02    .3684 3.424 1e-05    58 86.14 30.69      .4122   
    total_train_updates  tpb  tps   ups  
                  14498 5974 8873 1.485

22:11:14 | time:6677s total_exs:229224 epochs:0.30
    clip  ctpb  ctps  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  loss    lr  ltpb  ltps   ppl  token_acc  \
       1  5849 15152 36.46  380             16384  18.14    .3858 3.375 1e-05 67.15   174 29.23      .4175   
    total_train_updates  tpb   tps   ups  
                  14525 5916 15326 2.591



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-13afe375ff26>", line 41, in <module>
    label_turns='both',  # https://parl.ai/docs/core/teachers.html#parlai.core.teachers.ConversationTeacher
  File "/usr/local/lib/python3.7/dist-packages/parlai/core/script.py", line 108, in main
    return cls._run_kwargs(kwargs)
  File "/usr/local/lib/python3.7/dist-packages/parlai/core/script.py", line 74, in _run_kwargs
    return cls._run_from_parser_and_opt(opt, parser)
  File "/usr/local/lib/python3.7/dist-packages/parlai/core/script.py", line 89, in _run_from_parser_and_opt
    return script.run()
  File "/usr/local/lib/python3.7/dist-packages/parlai/scripts/train_model.py", line 789, in run
    return self.train_loop.train()
  File "/usr/local/lib/python3.7/dist-packages/parlai/scripts/train_model.py", line 679, in train
  

KeyboardInterrupt: ignored

In [ ]:
!ls /content/drive/MyDrive/chatbot_model/